# Нейросеть для автодополнения текстов

## Бизнес-контекст и задача проекта
Представьте, что вы работаете в соцсетевом приложении, где пользователи постят короткие тексты. В продукте стоит задача — добавить возможность автодополнения текстов. Разработчики просят вас создать модель, которую можно запускать на мобильных устройствах. Для смартфонов есть значительные требования по оперативной памяти и скорости работы, так что важна легковесность модели. 

## Задача
**Работчики предоставили датасет с короткими текстовыми постами.**

Поэтапное описание задачи:
- Взять датасет от разработчиков, очистить его, подготовить для обучения модели.
- Реализовать и обучить модель на основе рекуррентных нейронных сетей.
- Замерить качество разработанной и обученной модели.
- Взять более «тяжёлую» предобученную модель из Transformers и замерить её качество.
- Проанализировать результаты и дать рекомендации разработчикам: стоит ли использовать лёгкую модель или лучше постараться поработать с ограничениями по памяти и использовать большую предобученную.


## Этап 1. Сбор и подготовка данных
### Загрузка датасета

In [1]:
# Инициализация библиотек
import pandas as pd
import re

# Устанавливаем максимальную ширину для всех колонок
pd.set_option('display.max_colwidth', None)  # None показывает весь текст

In [45]:
# чтение датасета
path_dataset = './data/tweets.txt'
with open(path_dataset, 'r') as f:
    raw = f.readlines()

# сохраняю датасет в датафрейм
df = pd.DataFrame({'tweet': raw})
df.head()

,tweet
0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D\n"
1,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!\n
2,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds\n
3,my whole body feels itchy and like its on fire \n
4,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. \n"


In [31]:
raw[:6]

["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D\n",
 "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!\n",
 '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds\n',
 'my whole body feels itchy and like its on fire \n',
 "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. \n",
 '@Kwesidei not the whole crew \n']

### Очистка и нормализация текста
- Привести к нижнему регистру;
- удалить ссылки, упоминания, эмодзи (по необходимости);
- заменить нестандартные символы;
- токенизировать текст.

In [ ]:
# очистка данных

# 2. Очистка текстов
def clean_text(text):
    # Удаление ссылок
    text = re.sub(r'http\S+', '', text)

    # Удаление упоминаний
    text = re.sub(r'@\w+', '', text)

    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip() 
    text = re.sub(r'\.(?:\s*\.){1,}', ' ', text)
    text = re.sub(r'!(?:\s*!){1,}', '!', text)
    text = re.sub(r'^-', '', text)


    text = text.lower()  # к нижнему регистру
    return text



In [47]:
df['tweet'] = df['tweet'].apply(clean_text)
df.head()

,tweet
0,"awww, that's a bummer. you shoulda got david carr of third day to do it. ;d"
1,is upset that he can't update his facebook by texting it and might cry as a result school today also. blah!
2,i dived many times for the ball. managed to save 50% the rest go out of bounds
3,my whole body feels itchy and like its on fire
4,"no, it's not behaving at all. i'm mad. why am i here? because i can't see you all over there."


In [36]:
item_df = df.head(6)
item_df['tweet'].apply(clean_text)

0                                    awww, that's a bummer. you shoulda got david carr of third day to do it. ;d
1    is upset that he can't update his facebook by texting it and might cry as a result school today also. blah!
2                                 i dived many times for the ball. managed to save 50% the rest go out of bounds
3                                                                 my whole body feels itchy and like its on fire
4                  no, it's not behaving at all. i'm mad. why am i here? because i can't see you all over there.
5                                                                                             not the whole crew
Name: tweet, dtype: object

In [48]:
# Сохраняем очищенный текст в файл
df['tweet'].to_csv('./data/tweets_clean_text.txt', 
                         index=False, 
                         header=False, 
                         sep='\n',  # каждая строка - новый элемент
                         encoding='utf-8')